## Basic usage of Chatgpt API for paragraph summarization

### Library 

In [49]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

### API Key

In [50]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


### Connection 

In [51]:
openai = OpenAI()

### Scrap website for summarization

In [52]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        self.title = soup.title.string if soup.title else "No title found"

        paragraphs = soup.find_all("p")
        self.cleaned_paragraphs = str([p.text for p in paragraphs][6:22])   
        
        

### Construct system prompt & user prompt

In [53]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.cleaned_paragraphs
    return user_prompt

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

### Output

In [54]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [55]:
display_summary("https://en.wikipedia.org/wiki/History_of_tariffs_in_the_United_States")

# Summary of "History of tariffs in the United States"

The article details the development and evolution of tariffs in the United States from its independence in 1776 through the post-World War II era. Key points include:

- **Initial Framework**: The federal government's ability to impose tariffs began with the Constitution in 1789, leading to the Tariff Act of 1789, which introduced a 5% tariff on imports. The U.S. Revenue Cutter Service was established to enforce these tariffs.

- **Protectionism Advocacy**: Alexander Hamilton and other economists argued for protective tariffs to develop U.S. manufacturing industries, marking the U.S. as a significant advocate of protectionism compared to Britain's free trade principles.

- **Infant Industry Argument**: Hamilton introduced the concept of "infant industries," advocating temporary protection against foreign competition to allow these industries to grow.

- **Significant Tariff Changes**: The article chronicles how tariff rates increased during wartime, such as the War of 1812 and the Civil War, as well as political movements that shaped tariff policy, like the American System promoted by Henry Clay.

- **Lifelong Use of Tariffs**: U.S. tariffs remained high from the 19th century into the early 20th century, with periods of significant protectionist measures influencing industrial growth and national economic performance.

- **Post-War Liberalization**: After World War II, the U.S. began to liberalize its trade policies, marking a shift from previous high tariff strategies.

Overall, the U.S. experience with tariffs is portrayed as fundamental to its industrial independence and economic growth throughout its early history. The article suggests that protectionist policies directly contributed to the nation's rapid industrial development up until the mid-20th century.